In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dataset = "arrows"
experiment_name = "arrows_datapairs_chamferreg"
saved_models_dir = os.path.join(os.path.dirname(os.getcwd()), "saved_models")
dataset_dir = os.path.join(saved_models_dir, dataset)
experiment_dir = os.path.join(dataset_dir, experiment_name)


In [ ]:
os.listdir(experiment_dir)

In [ ]:
def get_metrics_N(experiment_dir, dataset_name, extra_dimension, N_values, metric, seeds):
    equivariance_losses_N = []
    for N in N_values:
        equivariance_losses = []
        for seed in seeds:
            saved_data_dir = "D{}-D{}-Linfonce-ED{}-N{}-Mresnet-Aresnet-ID{}-S{}".format(dataset, dataset_name, extra_dimension, N, experiment_name, seed)
            files_dir = os.path.join(experiment_dir, saved_data_dir)
            if metric == "errors_rec_val":
                equivariance_losses.append(np.load(os.path.join(files_dir, metric+".npy"))[-1]/(64*64))
            else:
                equivariance_losses.append(np.load(os.path.join(files_dir, metric+".npy"))[-1])
        equivariance_losses = np.array(equivariance_losses)
        equivariance_losses_N.append(equivariance_losses)
        print(N,equivariance_losses.shape)
    return np.array(equivariance_losses_N)
    
def get_metrics_chamferreg(experiment_dir, dataset_name, extra_dimension, chamfers, metric, seeds, N):
    equivariance_losses_N = []
    for chamfer in chamfers:
        equivariance_losses = []
        for seed in seeds:
            saved_data_dir = "D{}-D{}-Linfonce-ED{}-N{}-Mresnet-Aresnet-ID{}-S{}-CHAMFER{}".format(dataset, dataset_name, extra_dimension, N, experiment_name, seed, chamfer)
            files_dir = os.path.join(experiment_dir, saved_data_dir)
            if metric == "errors_rec_val":
                equivariance_losses.append(np.load(os.path.join(files_dir, metric+".npy"))[-1]/(64*64))
            else:
                equivariance_losses.append(np.load(os.path.join(files_dir, metric+".npy"))[-1])
        equivariance_losses = np.array(equivariance_losses)
        equivariance_losses_N.append(equivariance_losses)
        print(chamfer,equivariance_losses.shape)
    return np.array(equivariance_losses_N)

    

# N Values

In [ ]:
dataset = "symmetric_solids"
experiment_name = "arrows_datapairs_weighted1"
# experiment_name = "arrows_datapairs_weighted1chamfer1"
saved_models_dir = os.path.join(os.path.dirname(os.getcwd()), "saved_models")
dataset_dir = os.path.join(saved_models_dir, dataset)
experiment_dir = os.path.join(dataset_dir, experiment_name)

In [ ]:
print(os.listdir(experiment_dir))

In [ ]:
dataset_name = "1_c_2_c_3_c_4_c_5_c"
dataset_name = "1_2_3_4_5"
dataset_name = "tetrahedron cube icosahedron"
dataset_name = "tetrahedron"
extra_dimension = 0
N = 1
N_values = [1,3,4,5,6,7,8,9]
N_values = np.arange(1, 11)
N_values = np.arange(1, 21)
seed = 17
seeds = [17, 58, 19, 42, 28]
metric = "errors_rec_val"
metric = "errors_hitrate"
metric = "equiv_val"
metric = "entropy_val"
metric_title_dict = {"errors_rec_val": "Reconstruction", "equiv_val": "Equivariance", "errors_hitrate":"Hit-Rate", "entropy_val": "Discrete Entropy"}

equivariance_losses_N = get_metrics_N(experiment_dir, dataset_name, extra_dimension, N_values, metric, seeds)
std_equiv_losses = np.std(equivariance_losses_N, axis = -1)
q1,q3 = np.percentile(equivariance_losses_N, q=[25, 75], axis =-1)
mean_equiv_losses = np.mean(equivariance_losses_N, axis = -1)

In [ ]:
save_dir = "./"+dataset_name
os.makedirs(save_dir, exist_ok = True)

In [ ]:
fig , ax = plt.subplots(1,1)
ax.fill_between(N_values, mean_equiv_losses-std_equiv_losses, mean_equiv_losses+std_equiv_losses, color='k', alpha=.2, edgecolor="b")
ax.plot(N_values, mean_equiv_losses, ".-", markersize=10)
ax.set_xticks(N_values)
ax.set_ylabel(metric_title_dict[metric], fontsize = 12.5)
ax.set_xlabel(r"$N$", fontsize = 12.5)
# ax.set_yscale('log')
ax.set_xlim([N_values[0], N_values[-1]])
ax.set_title(r"{} vs. $N$".format(metric_title_dict[metric],dataset_name))
ax.grid()
fig.savefig(os.path.join(save_dir, metric+".png"))
np.save(os.path.join(save_dir, metric+"_mean.npy"), mean_equiv_losses)
np.save(os.path.join(save_dir, metric+"_std.npy"), std_equiv_losses)

# Chamfer

In [ ]:
dataset = "arrows"
experiment_name = "arrows_datapairs_chamferreg"
saved_models_dir = os.path.join(os.path.dirname(os.getcwd()), "saved_models")
dataset_dir = os.path.join(saved_models_dir, dataset)
experiment_dir = os.path.join(dataset_dir, experiment_name)

In [ ]:
dataset_name = "1_c_2_c_3_c_4_c_5_c"
dataset_name = "1_2_3_4_5"
extra_dimension = 3
N = 5
chamfer_values = [0.001, 0.01, 0.1, 1.0, 5.0, 10.0]
seeds = [17, 58, 19, 42, 28]
seeds = [17, 58]
# seeds = [17]
metric = "errors_rec_val"
# metric = "errors_hitrate"
metric = "equiv_val"
# metric = "entropy_val"

equivariance_losses_N = get_metrics_chamferreg(experiment_dir, dataset_name, extra_dimension, chamfer_values, metric, seeds, N)

In [ ]:
std_equiv_losses = np.std(equivariance_losses_N, axis = -1)
q1,q3 = np.percentile(equivariance_losses_N, q=[25, 75], axis =-1)
mean_equiv_losses = np.mean(equivariance_losses_N, axis = -1)

In [ ]:
os.makedirs("./chamferreg", exist_ok=True)

In [ ]:
fig , ax = plt.subplots(1,1)
ax.fill_between(chamfer_values, mean_equiv_losses-std_equiv_losses, mean_equiv_losses+std_equiv_losses, color='k', alpha=.2, edgecolor="b")
ax.plot(chamfer_values, mean_equiv_losses, ".-", markersize=10)
ax.set_xticks(chamfer_values)
ax.set_ylabel(metric_title_dict[metric], fontsize = 12.5)
ax.set_xlabel(r"$\log\lambda$", fontsize = 12.5)
ax.set_xscale('log')
# ax.set_yscale('log')
ax.set_xlim([chamfer_values[0], chamfer_values[-1]])
ax.set_title(r"{} vs. $\log\lambda$".format(metric_title_dict[metric],dataset_name))
ax.grid()
fig.savefig(os.path.join("./chamferreg", metric+".png"))

In [ ]:
dataset = "arrows"
experiment_name = "arrows_datapairs_weighted1"
saved_models_dir = os.path.join(os.path.dirname(os.getcwd()), "saved_models")
dataset_dir = os.path.join(saved_models_dir, dataset)
experiment_dir = os.path.join(dataset_dir, experiment_name)

In [ ]:
print(os.listdir(dataset_dir))

In [ ]:
print(os.listdir(experiment_dir))

In [ ]:
def load_N_results(dataset, dataset_name, experiment_dir, extra_dimension, N_values, seeds, metric:str):
    results_N = []
    for N in N_values:
        results = []
        for seed in seeds:
            saved_data_dir = "D{}-D{}-Linfonce-ED{}-N{}-Mresnet-Aresnet-ID{}-S{}".format(dataset, dataset_name, extra_dimension, N, experiment_name, seed)
            files_dir = os.path.join(experiment_dir, saved_data_dir)
            if metric == "errors_rec_val":
                results.append(np.load(os.path.join(files_dir, metric+".npy"))/(64*64))
            else:
                results.append(np.load(os.path.join(files_dir, metric+".npy")))
        results = np.array(results)
        results_N.append(results)
        
    return np.array(results_N)
    
    

In [ ]:
results_list = []
dataset_names = ["1", "2", "3", "4", "5"]

extra_dimension = 0
N_values = np.arange(1, 11)
seeds = [17, 58, 19, 42, 28]
for dataset_name in dataset_names:
    results_dict = {
        "hitrate":{"metric_name": "errors_hitrate",
                   "plot_name": "Hit-Rate",
                  "N_values":[],
                   "mean_N":[],
                   "std_N":[]
                  },
        "equiv":{"metric_name": "equiv_val",
                 "plot_name": "Equivariance",
                  "N_values":[],
                   "mean_N":[],
                   "std_N":[]
                  },
        "reconstruction":{"metric_name": "errors_rec_val",
                 "plot_name": "Reconstruction",
                  "N_values":[],
                   "mean_N":[],
                   "std_N":[]
                  },
        "entropy":{"metric_name": "entropy_val",
                 "plot_name": "Entropy",
                  "N_values":[],
                   "mean_N":[],
                   "std_N":[]
                  },
    }
    for key in results_dict.keys():
        results_dict[key]["N_values"] = load_N_results(dataset, dataset_name, experiment_dir, extra_dimension, N_values, seeds,results_dict[key]["metric_name"])
        results_dict[key]["mean_N"] = np.mean(results_dict[key]["N_values"], axis=1)
        results_dict[key]["std_N"] = np.std(results_dict[key]["N_values"], axis=1)
    results_list.append(results_dict)


In [ ]:
os.makedirs("./single_arrows", exist_ok=True)

In [ ]:
fig , ax = plt.subplots(1,1)
def plot_N_results(N_values, mean, std, ax, label = None):
    ax.fill_between(N_values, mean-std, mean+std, alpha=.2, edgecolor="b")
    if label is not None:
        ax.plot(N_values, mean, ".-", label =label )
    else:
        ax.plot(N_values, mean, ".-")
    ax.set_xticks(N_values)
    ax.set_xlabel("N", fontsize=12.5)
    ax.set_xlim([N_values[0], N_values[-1]])
    return ax
metric = "hitrate"
for num_result, result in enumerate(results_list):
    ax = plot_N_results(N_values, result[metric]["mean_N"][:, -1], result[metric]["std_N"][:, -1], ax, label = dataset_names[num_result])
ax.legend(title="Stabilizers")
ax.set_ylabel(result[metric]["plot_name"], fontsize=12.5)
ax.grid()
if metric == "reconstruction":
    ax.set_ylim([0, 0.7])
fig.savefig(os.path.join("./single_arrows", metric+".png"))

    

# ax.set_ylabel("Equivariance Loss")
# ax.grid()
# ax.set_title("{} loss per value of N for {} arrows".format(metric.split("_")[1],dataset_name))


In [ ]:
a = np.array([[2,3], [4,5]])
unique = np.unique(a, axis=0)[0]
np.product(a == unique, axis = -1, dtype=bool)

# a = np.array([2,3,4,5])
# unique = np.unique(a, axis=0)[0]
# np.product(a == unique, axis = -1, dtype=bool)

In [ ]:
fig , ax = plt.subplots(1,1)
for num_equiv_loss, (mean_equiv_loss, std_equiv_loss) in enumerate(zip(mean_equiv_losses, std_equiv_losses)):
    ax.plot(epochs, mean_equiv_loss, label = str(num_equiv_loss + 1))
    # ax.fill_between(epochs, (mean_equiv_loss-std_equiv_loss), (mean_equiv_loss+std_equiv_loss), color='b', alpha=.1)
ax.set_xlabel("Epochs")
ax.legend()
ax.set_ylabel("Equivariance Loss")

In [ ]:
mean_per_n = mean_equiv_losses[:, -1]
std_per_n = std_equiv_losses[:, -1]
q1_per_n = q1[:,-1]
q3_per_n = q3[:, -1]
fig , ax = plt.subplots(1,1)
ax.fill_between(N_values, mean_per_n-std_per_n, mean_per_n+std_per_n, color='k', alpha=.2, edgecolor="b")
ax.plot(N_values, mean_per_n, ".-")
ax.set_xticks(N_values)
ax.set_ylabel("Equivariance Loss")
ax.set_xlabel("N")
ax.set_xlim([N_values[0], N_values[-1]])
ax.set_title("{} loss per value of N for {} arrows".format(metric.split("_")[1],dataset_name))
ax.grid()
# ax.errorbar(range(len(mean_per_n)), mean_per_n, yerr=std_per_n)

In [ ]:
mean_per_n = mean_equiv_losses[:, -1]
std_per_n = std_equiv_losses[:, -1]
fig , ax = plt.subplots(1,1)
ax.fill_between(N_values, (mean_per_n-std_per_n), (mean_per_n+std_per_n), color='b', alpha=.1)
ax.plot(N_values, mean_per_n, "-x")
ax.set_xticks(N_values)
ax.set_ylabel("Equivariance Loss")
ax.set_xlabel("N")
ax.set_xlim([N_values[0], N_values[-1]])
ax.set_title("{} loss per value of N for {} arrows".format(metric.split("_")[1],dataset_name))
ax.grid()

In [ ]:

for num_equiv_loss, (mean_equiv_loss, std_equiv_loss) in enumerate(zip(mean_equiv_losses, std_equiv_losses)):
    ax.plot(epochs, mean_equiv_loss, label = str(num_equiv_loss + 1))
    # ax.fill_between(epochs, (mean_equiv_loss-std_equiv_loss), (mean_equiv_loss+std_equiv_loss), color='b', alpha=.1)
ax.set_xlabel("Epochs")
ax.legend()
ax.set_ylabel("Equivariance Loss")